In [1]:
%load_ext autoreload
%autoreload 2
from innolux import *

In [28]:
class SealAnalysis:
    class Table:
        @staticmethod
        def generate_delta_table(df: pl.DataFrame, condition: str | None = None) -> pl.DataFrame:
            delta_gap = {
                "Condition": [],
                "ID": [],
                "Location": [],
                "Class": [],
                "Delta(um)": [],
            }
            
            for glass_id in df['ID'].unique():
                delta_gap['ID'] += [glass_id] * 8
                delta_gap['Class'] += ['first - max', 'max - avg'] * 4
                if condition is None:
                    cond = df.filter(pl.col("ID") == glass_id)[0,'Condition']
                else:
                    cond = df.filter(pl.col("ID") == glass_id)[0,condition]
                delta_gap['Condition'] += [cond] * 8
                
                for location in ['up', 'down', 'left', 'right']:
                    tmp_df = df.filter(
                        pl.col('ID') == glass_id,
                        pl.col('Location') == location,
                        # (pl.col('Measure Order') % 20 > 4) | (pl.col('Measure Order') % 20 == 0),
                        pl.col('RMS(%)') < 1,
                    )
                    first = tmp_df[0,3]
                    avg = tmp_df[-10:,3].mean()
                    max_ = tmp_df[:,3].max()
                    delta_gap['Location'] += [location] * 2
                    delta_gap['Delta(um)'] += [first - max_, max_ - avg]
                    
            
            return pl.DataFrame(delta_gap)
    class Plot:
        @staticmethod
        def around_gap(df: pl.DataFrame, **params):
            figs = []
            for cond in df['Condition'].unique():
                print(cond)
                figs.append(px.scatter(
                    df.filter(pl.col('Condition')==cond),
                    color_continuous_scale=px.colors.sequential.Jet,
                    # color_discrete_sequence=px.colors.sequential.Jet,
                    **params,
                ))
                figs[-1].update_layout(yaxis_range=(3.1, 3.5))
                figs[-1].show()
            return figs
        
        @staticmethod
        def around_delta(df: pl.DataFrame, **params):
            conds = df['Condition'].unique()
            fig = make_subplots(rows=1, cols=len(conds), shared_yaxes=True)
            for index, cond in enumerate(conds):
                print(cond)
                tmp_df = df.filter(pl.col('Condition')==cond)
                sub_fig = px.box(
                    tmp_df,
                    **params
                )
                for trace in sub_fig.data:
                    fig.add_trace(trace, row=1, col=index+1)
            return fig

In [6]:
import plotly.io as pio 
pio.renderers.default = "vscode"

In [12]:
around_gap = pl.read_excel(data_path / "T2 6628 BP 3500 A ZIX-7061 S-NP13 S-WB141" / "around_gap_summary_20250903.xlsx")
around_gap

ID,Measure Order,Location,Cell Gap(um),Dist to Edge(mm),RMS(%),Panel ID,Condition,Cross Area,GRADE
str,i64,str,f64,i64,f64,str,str,str,str
"""210FB5""",0,"""center""",3.149687,0,0.248483,"""660L2AP50FN210FB5""","""LCT-15-1098 + S-WB141""","""2320""","""NG"""
"""210FB5""",1,"""up""",3.189865,0,0.773582,"""660L2AP50FN210FB5""","""LCT-15-1098 + S-WB141""","""2320""","""NG"""
"""210FB5""",2,"""up""",3.195987,1,0.239258,"""660L2AP50FN210FB5""","""LCT-15-1098 + S-WB141""","""2320""","""NG"""
"""210FB5""",3,"""up""",3.191534,2,0.231841,"""660L2AP50FN210FB5""","""LCT-15-1098 + S-WB141""","""2320""","""NG"""
"""210FB5""",4,"""up""",3.185135,3,0.253972,"""660L2AP50FN210FB5""","""LCT-15-1098 + S-WB141""","""2320""","""NG"""
…,…,…,…,…,…,…,…,…,…
"""215C96""",76,"""right""",3.235699,15,0.237454,"""660L2AP50FN215C96""","""ZIX-7061XX + S-WB141""","""2418""","""G1"""
"""215C96""",77,"""right""",3.235278,16,0.236192,"""660L2AP50FN215C96""","""ZIX-7061XX + S-WB141""","""2418""","""G1"""
"""215C96""",78,"""right""",3.235292,17,0.238952,"""660L2AP50FN215C96""","""ZIX-7061XX + S-WB141""","""2418""","""G1"""


In [24]:
around_gap.filter(pl.col("Panel ID") == "660L2AP50FN215C96")[0,'Condition']

'ZIX-7061XX + S-WB141'

In [45]:
fig = SealAnalysis.Plot.around_gap(
    around_gap.filter(pl.col("RMS(%)") < 1), # RMS < 0.3 -> no large jump point
    x='Measure Order',
    y='Cell Gap(um)',
    color='GRADE',
)

ZIX-7061XX + S-WB141


ZIX-7061XX + S-NP13


LCT-15-1098 + S-WB141


In [42]:
df = around_gap.filter(pl.col("RMS(%)") < 0.3)
params = {
    "x": 'Measure Order',
    "y": 'Cell Gap(um)',
    "color": 'GRADE',
}

figs = []
for cond in df['Condition'].unique():
    print(cond)
    figs.append(px.scatter(
        df.filter(pl.col('Condition')==cond),
        color_continuous_scale=px.colors.sequential.Jet,
        # color_discrete_sequence=px.colors.sequential.Jet,
        **params,
    ))
    figs[-1].update_layout(yaxis_range=(3.1, 3.5))
    figs[-1].show()

LCT-15-1098 + S-WB141


ZIX-7061XX + S-NP13


ZIX-7061XX + S-WB141


In [39]:
around_gap


ID,Measure Order,Location,Cell Gap(um),Dist to Edge(mm),RMS(%),Panel ID,Condition,Cross Area,GRADE
str,i64,str,f64,i64,f64,str,str,str,str
"""210FB5""",0,"""center""",3.149687,0,0.248483,"""660L2AP50FN210FB5""","""LCT-15-1098 + S-WB141""","""2320""","""NG"""
"""210FB5""",1,"""up""",3.189865,0,0.773582,"""660L2AP50FN210FB5""","""LCT-15-1098 + S-WB141""","""2320""","""NG"""
"""210FB5""",2,"""up""",3.195987,1,0.239258,"""660L2AP50FN210FB5""","""LCT-15-1098 + S-WB141""","""2320""","""NG"""
"""210FB5""",3,"""up""",3.191534,2,0.231841,"""660L2AP50FN210FB5""","""LCT-15-1098 + S-WB141""","""2320""","""NG"""
"""210FB5""",4,"""up""",3.185135,3,0.253972,"""660L2AP50FN210FB5""","""LCT-15-1098 + S-WB141""","""2320""","""NG"""
…,…,…,…,…,…,…,…,…,…
"""215C96""",76,"""right""",3.235699,15,0.237454,"""660L2AP50FN215C96""","""ZIX-7061XX + S-WB141""","""2418""","""G1"""
"""215C96""",77,"""right""",3.235278,16,0.236192,"""660L2AP50FN215C96""","""ZIX-7061XX + S-WB141""","""2418""","""G1"""
"""215C96""",78,"""right""",3.235292,17,0.238952,"""660L2AP50FN215C96""","""ZIX-7061XX + S-WB141""","""2418""","""G1"""


In [32]:
fig = SealAnalysis.Plot.around_delta(
    SealAnalysis.Table.generate_delta_table(around_gap),
    x='Location',
    y='Delta(um)',
    color='Class',
)
fig.show()

ZIX-7061XX + S-NP13
LCT-15-1098 + S-WB141
ZIX-7061XX + S-WB141


In [34]:
df = SealAnalysis.Table.generate_delta_table(around_gap)
params = {
    "x":'Location',
    "y":'Delta(um)',
    "color":'Class',
}
conds = df['Condition'].unique()
fig = make_subplots(rows=1, cols=len(conds), shared_yaxes=True)
for index, cond in enumerate(conds):
    print(cond)
    tmp_df = df.filter(pl.col('Condition')==cond)
    sub_fig = px.box(
        tmp_df,
        **params
    )
    for trace in sub_fig.data:
        fig.add_trace(trace, row=1, col=index+1)
fig.show()

ZIX-7061XX + S-NP13
LCT-15-1098 + S-WB141
ZIX-7061XX + S-WB141
